In [ ]:
PROJECT = "era5-land-project"

In [ ]:
!gcloud auth login --project {PROJECT}


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=NQqH3SVlss2qZuHX8n6Sa93rtxSzrY&prompt=consent&token_usage=remote&access_type=offline&code_challenge=iNV7NPTfhVMwINW2qsJI3cEEBOmLezia1yL-tX-mGhU&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrA1j8FT1heQJZvlN1TOqgjBNPRQHD0JPQ8MGXwrJTK3barxPi-zGKj5w2Q6LkTN3g

You are now logged in as [szhanerz@gmail.com].
Your current project is 

In [ ]:
import datetime
import ee
import logging # to check task status when exporting images
import math


In [ ]:
ee.Authenticate()
ee.Initialize(project=PROJECT)

# OLD: Calculate Monthly Max Temperature, Total Precipitation, and Degree Days
monthly_aggregation.py

In [ ]:


# # Define functions for aggregation
# def monthly_max_temp(year, month):
#     start_date = ee.Date.fromYMD(year, month, 1)
#     end_date = start_date.advance(1, 'month')
#     era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
#     monthly_data = era_hourly.filterDate(start_date, end_date).select('temperature_2m')
#     monthly_max = monthly_data.max().set('system:time_start', start_date.millis())
#     return monthly_max

# def monthly_total_precip(year, month):
#     start_date = ee.Date.fromYMD(year, month, 1)
#     end_date = start_date.advance(1, 'month')
#     era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
#     monthly_data = era_hourly.filterDate(start_date, end_date).select('total_precipitation')
#     monthly_total = monthly_data.sum().set('system:time_start', start_date.millis())
#     return monthly_total

# def degree_days(year, month, base_temp):
#     start_date = ee.Date.fromYMD(year, month, 1)
#     end_date = start_date.advance(1, 'month')
#     era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
#     monthly_data = era_hourly.filterDate(start_date, end_date).select('temperature_2m')
#     degree_days = monthly_data.map(lambda img: img.subtract(base_temp).max(0)).sum().set('system:time_start', start_date.millis())
#     return degree_days

# def generate_monthly_aggregates(years, base_temp=0):
#     monthly_aggregates = []
#     for year in years:
#         for month in range(1, 13):
#             monthly_max = monthly_max_temp(year, month)
#             monthly_precip = monthly_total_precip(year, month)
#             degree_day = degree_days(year, month, base_temp)
#             combined = monthly_max.addBands(monthly_precip.rename('total_precip')).addBands(degree_day.rename('degree_days_above_0'))
#             combined = combined.set('year', year).set('month', month)
#             monthly_aggregates.append(combined)
#     return ee.ImageCollection(monthly_aggregates)

# # Define the years for aggregation
# years = [2022, 2023]

# # Generate monthly aggregates
# monthly_aggregates_ic = generate_monthly_aggregates(years)


# Updated: Generic Aggregate function for different types of temperature stats

In [ ]:
def aggregate_monthly_data(year, month, band_name, operation, base_temp=0):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
    monthly_data = era_hourly.filterDate(start_date, end_date).select(band_name)

    if operation == 'max':
        aggregated_data = monthly_data.max()
    elif operation == 'min':
        aggregated_data = monthly_data.min()
    elif operation == 'mean':
        aggregated_data = monthly_data.mean()
    elif operation == 'sum':
        aggregated_data = monthly_data.sum()
    elif operation == 'degree_days':
        aggregated_data = monthly_data.map(lambda img: img.subtract(base_temp).max(0)).sum()
    else:
        raise ValueError(f"Unsupported operation: {operation}")

    # Convert temperature from Kelvin to Celsius if the band name includes 'temperature'
    if 'temperature' in band_name:
        aggregated_data = aggregated_data.subtract(273.15)

    return aggregated_data.set('system:time_start', start_date.millis())

In [ ]:
def generate_monthly_aggregates(years, base_temp=0):
    monthly_aggregates = []
    for year in years:
        for month in range(1, 13):
            monthly_max = aggregate_monthly_data(year, month, 'temperature_2m', 'max')
            monthly_min = aggregate_monthly_data(year, month, 'temperature_2m', 'min')
            monthly_mean = aggregate_monthly_data(year, month, 'temperature_2m', 'mean')
            monthly_precip = aggregate_monthly_data(year, month, 'total_precipitation', 'sum')
            degree_day = aggregate_monthly_data(year, month, 'temperature_2m', 'degree_days', base_temp)

            year_band = ee.Image.constant(year).rename('year').toFloat()
            month_band = ee.Image.constant(month).rename('month').toFloat()

            combined = monthly_max.rename('max_temp') \
                                  .addBands(monthly_min.rename('min_temp')) \
                                  .addBands(monthly_mean.rename('mean_temp')) \
                                  .addBands(monthly_precip.rename('total_precip')) \
                                  .addBands(degree_day.rename('degree_days_above_0')) \
                                  .addBands(year_band) \
                                  .addBands(month_band)

            monthly_aggregates.append(combined)
    return ee.ImageCollection(monthly_aggregates)



In [ ]:
monthly_aggregates_ic = generate_monthly_aggregates([2022, 2023])
print(monthly_aggregates_ic.size().getInfo())
print(monthly_aggregates_ic.first().bandNames().getInfo())
print(monthly_aggregates_ic.first().getInfo())

24
['max_temp', 'min_temp', 'mean_temp', 'total_precip', 'degree_days_above_0', 'year', 'month']
{'type': 'Image', 'bands': [{'id': 'max_temp', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'min_temp', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'mean_temp', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'total_precip', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'degree_days_above_0', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'year', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'month', 'data_type': {'type': 'PixelType

Main Changes by July 27, 2024

- ***Year and Month as Bands***: The year and month are added as bands to each image using ee.Image.constant(year).rename('year') and ee.Image.constant(month).rename('month'). This ensures that they are included in the exported data.
- ***Metadata Handling for Export***: The export function now uses reduceRegion with ee.Reducer.first() to extract the year and month from the image bands instead of using metadata. This approach ensures that these values are correctly extracted from the bands added to the image.
- ***Error Handling***: The function skips the export if year or month is missing, with a clear message indicating why.






# Exporitng aggregates to GEE as ImageCollection Asset

In [ ]:
# # Function to export each image in the collection to an asset
# def export_to_asset(image, index, asset_id_prefix):
#     image_id = ee.Number(index).format().getInfo()
#     asset_id = f"{asset_id_prefix}_{image_id}"
#     description = f"Export_monthly_aggregates_{image_id}"
#     task = ee.batch.Export.image.toAsset(
#         image=image,
#         description=description,
#         assetId=asset_id,
#         scale=1000,
#         region=ee.Geometry.Rectangle([-180, -90, 180, 90])
#     )
#     task.start()
#     print(f"Exporting {description} to {asset_id}")

# # Define the asset ID prefix
# asset_id_prefix = f'projects/{PROJECT}/assets/monthly_aggregates_22_23'

# # Export each image in the collection
# image_list = monthly_aggregates_ic.toList(monthly_aggregates_ic.size())
# for i in range(image_list.size().getInfo()):
#     export_to_asset(ee.Image(image_list.get(i)), i, asset_id_prefix)


# Exporting monthly aggregates to Google Drive in GEOTIFF format

Key Fix:
- Point Geometry for reduceRegion: The reduceRegion function now includes a point parameter to define a specific location for reduction. This resolves the issue of working with unbounded images by specifying a point on the globe.

In [ ]:
# # Function to export each image in the collection to Google Drive
# def export_to_drive(image, description_prefix, folder):
#     # Define a point or a small region to reduce the image over
#     point = ee.Geometry.Point([0, 0])  # Arbitrarily chosen point
#     year = image.select('year').reduceRegion(ee.Reducer.first(), point, 1000).get('year').getInfo()
#     month = image.select('month').reduceRegion(ee.Reducer.first(), point, 1000).get('month').getInfo()
#     if year is None or month is None:
#         print(f"Skipping export for image due to missing year or month bands.")
#         return
#     description = f"{description_prefix}_{int(year)}_{int(month):02d}"
#     export_params = {
#         'image': image,
#         'description': description,
#         'folder': folder,
#         'scale': 1000,
#         'region': ee.Geometry.Rectangle([-180, -90, 180, 90]),
#         'fileFormat': 'GeoTIFF'
#     }
#     task = ee.batch.Export.image.toDrive(**export_params)
#     task.start()
#     print(f"Exporting {description} to Google Drive")

# # Define the years for aggregation
# years = [2022, 2023]

# # Generate monthly aggregates
# monthly_aggregates_ic = generate_monthly_aggregates(years)

# # Define the export parameters
# description_prefix = 'monthly_aggregates'
# folder = 'export_era5_monthly_2_years_27jul'

# # Export each image in the collection
# image_list = monthly_aggregates_ic.toList(monthly_aggregates_ic.size())
# for i in range(image_list.size().getInfo()):
#     export_to_drive(ee.Image(image_list.get(i)), description_prefix, folder)

In [ ]:
# # Function to export each image in the collection to Google Drive
# def export_to_drive(image, description_prefix, folder):
#     year = image.get('year').getInfo()
#     month = image.get('month').getInfo()
#     if year is None or month is None:
#         print(f"Skipping export for image due to missing year or month metadata.")
#         return
#     description = f"{description_prefix}_{year}_{int(month):02d}"
#     export_params = {
#         'image': image,
#         'description': description,
#         'folder': folder,
#         'scale': 1000,
#         'region': ee.Geometry.Rectangle([-180, -90, 180, 90]),
#         'fileFormat': 'GeoTIFF'
#     }
#     task = ee.batch.Export.image.toDrive(**export_params)
#     task.start()
#     print(f"Exporting {description} to Google Drive")

# # Define the export parameters
# description_prefix = 'monthly_aggregates'
# folder = 'export_era5_monthly_2years_aggregates'
# #
# # Export each image in the collection
# image_list = monthly_aggregates_ic.toList(monthly_aggregates_ic.size())
# for i in range(image_list.size().getInfo()):
#     export_to_drive(ee.Image(image_list.get(i)), description_prefix, folder)




> Link to output:
> https://drive.google.com/drive/folders/1Hr1q8DE10YvcwEFdUzyzIMLEBilLbbsa?usp=sharing



# Exporting in CSV

- used zonal_stats function for computing statistics over an image collection based on a feature collection.
- Coordinates FeatureCollection: Replaced this with actual feature collection.

- Export Parameters: The Export.table.toDrive function exports the resulting feature collection to Google Drive in CSV format.

In [ ]:
coordinates = ee.FeatureCollection("projects/era5-land-project/assets/itrdb_locations_unique_with_duplicate_lat_lon_info")


In [ ]:
def zonal_stats(ic, fc, params=None):
    _params = {
        'reducer': ee.Reducer.mean(),
        'scale': None,
        'crs': None,
        'bands': None,
        'bandsRename': None,
        'imgProps': None,
        'imgPropsRename': None,
        'datetimeName': 'datetime',
        'datetimeFormat': 'YYYY-MM-dd HH:mm:ss'
    }
    if params:
        for param in params:
            _params[param] = params[param] or _params[param]

    img_rep = ic.first()
    non_system_img_props = ee.Feature(None).copyProperties(img_rep).propertyNames()
    if not _params['bands']:
        _params['bands'] = img_rep.bandNames()
    if not _params['bandsRename']:
        _params['bandsRename'] = _params['bands']
    if not _params['imgProps']:
        _params['imgProps'] = non_system_img_props
    if not _params['imgPropsRename']:
        _params['imgPropsRename'] = _params['imgProps']

    def map_function(img):
        img = ee.Image(img.select(_params['bands'], _params['bandsRename']))
        img = img.set(_params['datetimeName'], img.date().format(_params['datetimeFormat']))
        img = img.set('timestamp', img.get('system:time_start'))
        props_from = ee.List(_params['imgProps']).cat(ee.List([_params['datetimeName'], 'timestamp']))
        props_to = ee.List(_params['imgPropsRename']).cat(ee.List([_params['datetimeName'], 'timestamp']))
        img_props = img.toDictionary(props_from).rename(props_from, props_to)
        fc_sub = fc.filterBounds(img.geometry())
        return img.reduceRegions(
            collection=fc_sub,
            reducer=_params['reducer'],
            scale=_params['scale'],
            crs=_params['crs']
        ).map(lambda f: f.set(img_props))

    results = ic.map(map_function).flatten()
    return results


In [ ]:
# Define the years for aggregation
years = [2022, 2023]

# Generate monthly aggregates
monthly_aggregates_ic = generate_monthly_aggregates(years)


In [ ]:
# Define coordinates feature collection
coordinates = ee.FeatureCollection("projects/era5-land-project/assets/itrdb_locations_unique_with_duplicate_lat_lon_info")

# Apply zonal statistics to the monthly aggregates
params = {'reducer': ee.Reducer.mean(), 'scale': 1000}
ptsERA5monthly = zonal_stats(monthly_aggregates_ic, coordinates, params)

# Export to Google Drive
export_params = {
    'collection': ptsERA5monthly,
    'folder': 'export_era5_monthly_new',
    'description': "monthly_aggregates_export_2022_2023_temp",
    'fileFormat': 'CSV'
}

task = ee.batch.Export.table.toDrive(**export_params)
task.start()
print("Exporting monthly aggregates to Google Drive")

Exporting monthly aggregates to Google Drive


#### Link to Output:
https://drive.google.com/drive/folders/1KXZrEP5CPJFNo1z09smH03CDWKFQAEh0?usp=sharing

- complete
monthly_aggregates_export_2022_2023_temp